<a href="https://colab.research.google.com/github/seajai24/fantasy-football-2021/blob/main/Fantasy_Football_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
espn = pd.read_csv("ESPN.csv")
fbbcalc = pd.read_csv("FBBCalc.csv")
ring = pd.read_csv("Ringer.csv")
br = pd.read_csv("BR.csv")
sportnews = pd.read_csv("SportingNews.csv")
fantasypros = pd.read_csv("FantasyPros.csv")
position = pd.read_csv("Positions.csv")
jamey = pd.read_csv("JameyCBS.csv")
dave = pd.read_csv("DaveCBS.csv")
heath = pd.read_csv("HeathCBS.csv")
pff = pd.read_csv("PFF.csv")
score = pd.read_csv("theScore.csv")

In [ ]:
fantasypros['Player'] = fantasypros['Player'].str.rstrip()

In [ ]:
espn['ESPN Rank'] = espn['ESPN Rank'].astype(float)

In [ ]:
initials = []
for i in range(len(position['Player'])) :
  name = position['Player'][i].partition(" ")
  first_init = name[0]
  new_name = first_init[0] + ". " + name[2]
  initials.append({'Name': position['Player'][i], 'Player': new_name, 'Position': position['Position'][i]})
inits = pd.DataFrame(initials)

In [ ]:
inits_merge = pd.merge(left = inits, right = heath, how = 'left', on = ['Player', 'Position'])
inits_merge2 = pd.merge(left = inits_merge, right = dave, how = 'left', on = ['Player', 'Position'])
inits_merge3 = pd.merge(left = inits_merge2, right = jamey, how = 'left', on = ['Player', 'Position'])

inits_merge3 = inits_merge3[['Name', 'Jamey CBS Rank', 'Dave CBS Rank', 'Heath CBS Rank']]
inits_merge3 = inits_merge3.drop_duplicates(subset = 'Name')
inits_merge3 = inits_merge3.reset_index(drop = True)
inits_merge3.columns = ['Player', 'Jamey CBS Rank', 'Dave CBS Rank', 'Heath CBS Rank']

In [ ]:
#Fix messed up values from duplicates
#AJ Brown
inits_merge3.at[120, 'Jamey CBS Rank'] = 21
inits_merge3.at[120, 'Dave CBS Rank'] = 22
inits_merge3.at[120, 'Heath CBS Rank'] = 14

#Antonio Brown
inits_merge3.at[139, 'Jamey CBS Rank'] = 89
inits_merge3.at[139, 'Dave CBS Rank'] = 77
inits_merge3.at[139, 'Heath CBS Rank'] = 71

#Jamaal Williams
inits_merge3.at[86, 'Jamey CBS Rank'] = 97
inits_merge3.at[86, 'Dave CBS Rank'] = 96
inits_merge3.at[86, 'Heath CBS Rank'] = 84

#JaVonte Williams
inits_merge3.at[57, 'Jamey CBS Rank'] = 66
inits_merge3.at[57, 'Dave CBS Rank'] = 69
inits_merge3.at[57, 'Heath CBS Rank'] = 80

#Damien Williams
inits_merge3.at[76, 'Jamey CBS Rank'] = 180
inits_merge3.at[76, 'Dave CBS Rank'] = 156
inits_merge3.at[76, 'Heath CBS Rank'] = 187

In [ ]:
r1 = pd.merge(left = espn, right = ring, how = 'left', left_on = 'Player', right_on = 'Player')
r2 = pd.merge(left = r1, right = fbbcalc, how = 'left', left_on = 'Player', right_on = 'Player')
r3 = pd.merge(left = r2, right = br, how = 'left', left_on = 'Player', right_on = 'Player')
r4 = pd.merge(left = r3, right = sportnews, how = 'left', left_on = 'Player', right_on = 'Player')
r5 = pd.merge(left = r4, right = fantasypros, how = 'left', left_on = 'Player', right_on = 'Player')
r6 = pd.merge(left = r5, right = inits_merge3, how = 'left', left_on = 'Player', right_on = 'Player')
r7 = pd.merge(left = r6, right = pff, how = 'left', left_on = 'Player', right_on = 'Player')
r8 = pd.merge(left = r7, right = score, how = 'left', left_on = 'Player', right_on = 'Player')
rank = pd.merge(left = r8, right = position, how = 'left', left_on = 'Player', right_on = 'Player')

In [ ]:
rank['ADP'] = round(rank.mean(axis = 1),2)
rank = rank.sort_values(by = 'ADP')
rank = rank[['Player', 'Position', 'ESPN Rank', 'Ringer Rank', 'FBB Calc Rank', 'BR Rank', 'Sporting News Rank', 'Fantasy Pros Rank', 'Jamey CBS Rank', 'Dave CBS Rank', 'Heath CBS Rank', 'PFF Rank', 'theScore Rank', 'ADP']]
rank = rank.reset_index(drop = True)

In [ ]:
rank['12 Team Round'] = 0
rank['12 Team Pick'] = 0
rank['10 Team Round'] = 0
rank['10 Team Pick'] = 0
for i in range(len(rank)) :
  rank['12 Team Round'][i] = (i // 12) + 1
  rank['12 Team Pick'][i] = (i+1) % 12
  rank['10 Team Round'][i] = (i // 10) + 1
  rank['10 Team Pick'][i] = (i+1) % 10

rank['12 Team Pick'] = rank['12 Team Pick'].replace(0,12)
rank['10 Team Pick'] = rank['10 Team Pick'].replace(0,10)

In [ ]:
rb, wr, te, qb, k, dst = 0, 0, 0, 0, 0, 0
rank['Position Rank'] = 'a'
for i in range(len(rank)) :
  if rank['Position'][i] == 'RB' :
    rb = rb + 1
    rank['Position Rank'][i] = 'RB{}'.format(rb)
  elif rank['Position'][i] == 'WR' :
    wr = wr + 1
    rank['Position Rank'][i] = 'WR{}'.format(wr)
  elif rank['Position'][i] == 'TE' :
    te = te + 1
    rank['Position Rank'][i] = 'TE{}'.format(te)
  elif rank['Position'][i] == 'QB' :
    qb = qb + 1
    rank['Position Rank'][i] = 'QB{}'.format(qb)
  elif rank['Position'][i] == 'K' :
    k = k + 1
    rank['Position Rank'][i] = 'K{}'.format(k)
  elif rank['Position'][i] == 'D/ST' :
    dst = dst + 1
    rank['Position Rank'][i] = 'D/ST{}'.format(dst)
  else :
    rank['Position Rank'][i] = 'Unknown'

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
rank.head(10)

In [ ]:
#Find rows with NAs
rank[rank.isnull().any(axis=1)]

In [ ]:
simple12 = rank[['Player', 'Position', 'ADP', '12 Team Round', '12 Team Pick', 'Position Rank']]

In [ ]:
simple12[(simple12['12 Team Round'] >= 4) & (simple12['12 Team Round'] <= 8)]

In [ ]:
rank.to_csv('Fantasy_Football_2021_Draft_Rank.csv', index = False)
!cp Fantasy_Football_2021_Draft_Rank.csv "drive/My Drive/"

In [ ]:
simple12.to_csv('Simple_12_Team_Rankings.csv', index = False)
!cp Simple_12_Team_Rankings.csv "drive/My Drive/"